# Case Study 3 : Patient Drug Review

Tha Songtan Suppasadid<br>
Nichakan Udomlerkchai<br>
Peeranon Posawang<br>
<br>
Last edited: 2/28/2020 9:17 PM<br>
Helsinki Metropolia University of Applied Science


## Background

## Data

In [19]:
import time # for timing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from sklearn.model_selection import train_test_split # preprocessing datasets
from tensorflow.keras.preprocessing.text import Tokenizer # text preprocessing
from tensorflow.keras.preprocessing.sequence import pad_sequences # text preprocessing
from tensorflow.keras.models import Sequential # modeling neural networks
from tensorflow.keras.layers import Dense, Activation # layers for neural networks
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score # final metrics
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers,models,optimizers,metrics
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding,LSTM,TimeDistributed,Activation, Dropout
from tensorflow.keras.models import Model

In [2]:
train = pd.read_csv('data/drugsComTrain_raw.csv')

In [3]:
train.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37


In [4]:
test = pd.read_csv('data/drugsComTest_raw.csv')

In [5]:
test.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [6]:
train.shape

(161297, 7)

In [7]:
test.shape

(53766, 7)

In [8]:
%%time
# Tokenize the text
samples = train['review']
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(samples)
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(samples)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 51429 unique tokens.
Wall time: 30.1 s


In [9]:
data = pad_sequences(sequences, maxlen=200)

In [10]:
%%time
# Create three categories
# label = 4, when rating == 10
# label = 3, when rating == 8...9
# label = 2, when rating = 5..7
# label = 1, when rating = 2..4
# label = 0, when rating = 1
labels = train['rating'].values
for i in range(len(labels)):
    x = labels[i]
    if x == 10:
        labels[i] = 4
    elif x >= 8:
        labels[i] = 3
    elif x >= 5:
        labels[i] = 2
    elif x >= 2:
        labels[i] = 1
    else:
        labels[i] = 0

Wall time: 182 ms


In [11]:
from tensorflow.keras.utils import to_categorical

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (161297, 200)
Shape of label tensor: (161297, 5)


In [12]:
VALIDATION_SPLIT = 0.25

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

# Model 1

In [ ]:
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

In [ ]:
embedding_layer = Embedding(5000,
                            100,
                            input_length=200,
                            trainable=True)

sequence_input = Input(shape=(200,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(5, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

model.summary()

In [ ]:
%%time
history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

In [ ]:
# Plot the accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
e = np.arange(len(acc)) + 1

plt.plot(e, acc, label = 'train')
plt.plot(e, val_acc, label = 'validation')
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

plt.figure()

plt.plot(e, loss, label = 'train')
plt.plot(e, val_loss, label = 'validation')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

plt.show()

In [ ]:
# Find the predicted values for the validation set
y_pred = np.argmax(model.predict(x_val), axis = 1)
y_true = np.argmax(y_val, axis = 1)

In [ ]:
# Calculate the classification report
cr = classification_report(y_true, y_pred)
print(cr)

In [ ]:
cm = confusion_matrix(y_true, y_pred).T
print(cm)

In [ ]:
# Calculate the cohen's kappa, both with linear and quadratic weights
k = cohen_kappa_score(y_true, y_pred)
print(f"Cohen's kappa (linear)    = {k:.3f}")
k2 = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
print(f"Cohen's kappa (quadratic) = {k2:.3f}")

# Model 2

In [14]:
len(x_train)

120973

In [20]:
vocabulary = 50000
hidden_size = 128
num_steps = 200

# model = models.Sequential()
# model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
# model.add(LSTM(hidden_size,activation='tanh', return_sequences=True))
# model.add(LSTM(hidden_size,activation='tanh', return_sequences=True))
# #model.add(LSTM(hidden_size, return_sequences=True))
# #model.add(TimeDistributed(Dense(vocabulary)))
# model.add(GlobalMaxPooling1D())
# model.add(layers.Dense(64,activation='tanh'))
# model.add(layers.Dense(5,activation='softmax'))
# model.summary()

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
# model.add(layers.SpatialDropout1D(0.3))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,return_sequences=False))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 128)          6400000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 325       
Total params: 6,540,165
Trainable params: 6,540,165
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
%%time
history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 120973 samples, validate on 40324 samples
Epoch 1/10
120973/120973 [==============================] - 1065s 9ms/sample - loss: 1.2783 - acc: 0.4340 - val_loss: 1.2187 - val_acc: 0.4761
Epoch 2/10
120973/120973 [==============================] - 1262s 10ms/sample - loss: 1.1668 - acc: 0.4908 - val_loss: 1.1424 - val_acc: 0.5053
Epoch 3/10
120973/120973 [==============================] - 1210s 10ms/sample - loss: 1.1073 - acc: 0.5197 - val_loss: 1.0985 - val_acc: 0.5268
Epoch 4/10
120973/120973 [==============================] - 1327s 11ms/sample - loss: 1.0646 - acc: 0.5378 - val_loss: 1.0807 - val_acc: 0.5335
Epoch 5/10
120973/120973 [==============================] - 1344s 11ms/sample - loss: 1.0233 - acc: 0.5564 - val_loss: 1.0677 - val_acc: 0.5433
Epoch 6/10
120973/120973 [==============================] - 1313s 11ms/sample - loss: 0.9865 - acc: 0.5718 - val_loss: 1.0460 - val_acc: 0.5522
Epoch 7/10
120973/120973 [==============================] - 1319s 11ms/sample - loss: 

In [17]:
history.history.keys()

NameError: name 'history' is not defined

In [ ]:
# Plot the accuracy and loss
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
e = np.arange(len(acc)) + 1

plt.plot(e, acc, label = 'train')
plt.plot(e, val_acc, label = 'validation')
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

plt.figure()

plt.plot(e, loss, label = 'train')
plt.plot(e, val_loss, label = 'validation')
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.grid()
plt.legend()

plt.show()

In [ ]:
# Find the predicted values for the validation set
y_pred = np.argmax(model.predict(x_val), axis = 1)
y_true = np.argmax(y_val, axis = 1)

In [ ]:
y_pred

In [ ]:
# Calculate the classification report
cr = classification_report(y_true, y_pred)
print(cr)

In [ ]:
cm = confusion_matrix(y_true, y_pred).T
print(cm)

In [ ]:
# Calculate the cohen's kappa, both with linear and quadratic weights
k = cohen_kappa_score(y_true, y_pred)
print(f"Cohen's kappa (linear)    = {k:.3f}")
k2 = cohen_kappa_score(y_true, y_pred, weights = 'quadratic')
print(f"Cohen's kappa (quadratic) = {k2:.3f}")